# Hugging Face 요약 모델 불러오기

In [3]:
!pip install -U transformers tokenizers huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0


### digit82/kobart-summarization 모델
- KoBART 기반 요약 모델
- 입력 최대 길이는 512 토큰 (1024 토큰 입력 불가)
- 출력 요약 길이는 기본적으로 30~150 토큰
Beam Search (num_beams=5)로 요약 생성 성능 향상 가능

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

SUMMARIZATION_MODEL = "digit82/kobart-summarization"

tokenizer_summarization = AutoTokenizer.from_pretrained(SUMMARIZATION_MODEL, use_fast=False)
model_summarization = AutoModelForSeq2SeqLM.from_pretrained(SUMMARIZATION_MODEL).to("cuda")

print("한국어 요약 모델 로드 완료!")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

한국어 요약 모델 로드 완료!


# Google Drive에 저장된 데이터 로드

In [2]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [23]:
import pandas as pd

DATA_PATH = "/content/drive/MyDrive/Project/project-AI-Pet-Robot/total_kor_counsel_bot_clean.csv"
df = pd.read_csv(DATA_PATH)

print(f"데이터 로드 완료! 총 {len(df)} 개 데이터")

데이터 로드 완료! 총 13204 개 데이터


# Hugging Face 요약 모델을 활용한 자동 요약 함수
- 512 토큰을 초과하는 경우 자동으로 요약하여 길이를 줄임
- KoBART 요약 모델을 사용하여 길이가 긴 문장을 자동으로 요약
- 1024 토큰까지 받아서 50~150 토큰 범위로 요약
- Beam Search (num_beams=4)로 좀 더 좋은 요약 생성

In [34]:
def summarize_text(text):
    """
    상담 내용을 자연스럽게 요약하는 최적화된 함수
    """
    prompt = (
    "다음 상담 내용을 핵심을 유지하면서 자연스럽게 요약하세요. "
    "불필요한 반복과 중복 표현을 제거하되, 중요한 정보는 유지하세요. "
    "대화의 흐름을 부드럽게 정리하고, 공감하는 느낌을 살리세요. "
    "문장을 지나치게 축약하지 말고, 자연스럽게 연결하세요. "
    "강조할 부분은 유지하되, 명령조의 문장은 피하고 부드러운 표현을 사용하세요. "
    "요약이 너무 짧거나 길어지지 않도록 적절한 길이를 유지하세요. "
    "단순한 정보 나열이 아닌, 대화의 흐름을 반영해 정리하세요. "
    "광고성 표현, 불필요한 기술 용어, 특정 브랜드 언급을 하지 마세요. "
    "문장이 중간에서 끊기지 않도록 하세요. "
    "상담자의 말투를 유지하되, 결론을 명확하게 제시하세요. "
    "이전 문장의 의미를 반복하지 않도록 하고, 한 문장에서 여러 의미가 겹치지 않도록 하세요. "
    "문장의 흐름이 부드럽도록 접속사를 적절히 활용하세요. "
    "상담 내용의 감정적 톤을 유지하되, 너무 극단적이거나 과장된 표현은 피하세요. "
    "대화를 나누는 느낌을 살려 요약하되, 지나치게 구어체로 흐르지 않도록 하세요. "
    "요약은 반드시 원문의 의미를 유지해야 하며, 새로운 정보를 추가하지 마세요. "
    "반복되는 조언보다는 핵심적인 해결 방법을 강조하세요. "
    "문장이 부자연스럽거나 어색하게 끝나지 않도록 하세요. "
    "프롬프트 내용을 요약 시 넣지 마세요."
    "원문의 논리적인 흐름을 유지하며, 너무 기계적인 축약이 되지 않도록 하세요.\n\n"
    f"본문: {text}\n\n"
    "요약:"
)



    inputs = tokenizer_summarization(
        prompt, return_tensors="pt", max_length=512, truncation=True
    ).to("cuda")

    summary_ids = model_summarization.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=220,  # 기존 180 → 220 (더 자연스러운 표현 유지)
        min_length=150,  # 기존 130 → 150 (너무 짧아지는 것 방지)
        length_penalty=1.4,  # 기존 1.6 → 1.4 (자연스럽게 압축)
        num_beams=5,  # 기존 6 → 5 (중복 제거하면서도 자연스럽게)
        repetition_penalty=2.2,  # 기존 2.5 → 2.2 (반복 최소화)
        no_repeat_ngram_size=4,  # 기존 5 → 4 (동일 구절 반복 방지)
        do_sample=False,  # 기존 False 유지 (정확한 요약 생성)
        temperature=0,  # 기존 None → 0 (랜덤성 제거)
        early_stopping=True  # 불필요한 문장 생성을 방지
    )

    summary = tokenizer_summarization.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [35]:
test_text = """청소년 상담에서 자주 등장하는 문제로는 대인 관계, 학업 스트레스, 가족 문제, 자존감 부족 등이 있습니다.
많은 청소년들은 자신이 어떤 고민을 가지고 있는지조차 인식하지 못하는 경우가 많으며, 이를 해결하는 과정에서 외부의 도움이 필요합니다.
따라서 효과적인 청소년 상담 모델을 개발하는 것은 매우 중요한 일입니다. 이를 위해 AI 챗봇을 활용할 수 있으며, 자연어 처리를 통해 감정 분석을 수행하고,
적절한 피드백을 제공할 수 있도록 설계해야 합니다. 또한, 사용자 경험을 향상시키기 위해 상담 데이터를 지속적으로 학습하고 업데이트하는 과정이 필요합니다."""

summary_result = summarize_text(test_text)
print("요약 결과:", summary_result)
print(len(summary_result))


요약 결과: 다음 상담 내용을 핵심을 유지하면서 자연스럽게 요약하고, 불필요한 반복과 중복 표현을 제거하되, 중요한 정보는 유지하듯이 대화의 흐름을 부드럽게 정리하고 공감하는 느낌을 살리도록 유도해야 하며, 대화를 나누는 느낌을 살려 요약하되, 지나치게 극단적이거나 과장된 표현은 피하며 적절한 길이를 유지해야 한다고 강조했다. ﻿

요약: 청소년 상담에서 자주 등장하는 문제로는 대인 관계, 학업 스트레스, 가족 문제, 자존감 부족 등이 있으며 이러한 문제들을 해결하기 위해 외부의 도움이 필요함은 매우 중요한 일이고 AI 챗봇을 활용할 수 있는 자연어 처리를 통해 감정 분석을 수행하고, 적절한 피드백을 제공할 수 있도록 설계해야 한다.  가맹한 조언보다는 핵심적인 해결 방법을 강조하여 도움을 줄 수 있다.
391


# 데이터 요약 적용 (512 토큰 초과하는 문장만 요약)

In [36]:
# 토큰 길이 확인 함수
def get_token_length(text):
    return len(tokenizer_summarization.encode(text, add_special_tokens=True))

# 긴 문장 요약 적용
df["input_length"] = df["input"].apply(get_token_length)
df["output_length"] = df["output"].apply(get_token_length)

# 요약이 필요한 데이터 개수 확인
num_long_inputs = len(df[df["input_length"] > 512])
num_long_outputs = len(df[df["output_length"] > 512])

print(f"512 토큰 초과 질문 개수: {num_long_inputs}개")
print(f"512 토큰 초과 답변 개수: {num_long_outputs}개")

# 512 토큰 초과하는 데이터 요약 적용 (수정된 .loc 방식)
df.loc[df["input_length"] > 512, "input"] = df.loc[df["input_length"] > 512, "input"].apply(summarize_text)
df.loc[df["output_length"] > 512, "output"] = df.loc[df["output_length"] > 512, "output"].apply(summarize_text)

# 다시 토큰 길이 확인
df["input_length"] = df["input"].apply(get_token_length)
df["output_length"] = df["output"].apply(get_token_length)

print(f"요약 후 512 토큰 이하 질문 개수: {len(df[df['input_length'] <= 512])}개")
print(f"요약 후 512 토큰 이하 답변 개수: {len(df[df['output_length'] <= 512])}개")

# 추가 검증: 요약 후에도 512 토큰 초과 데이터가 있는지 확인
num_long_inputs_after = len(df[df["input_length"] > 512])
num_long_outputs_after = len(df[df["output_length"] > 512])

print(f"요약 후 512 토큰 초과 질문 개수: {num_long_inputs_after}개")
print(f"요약 후 512 토큰 초과 답변 개수: {num_long_outputs_after}개")
print(f"요약 후 최대 질문 길이: {df['input_length'].max()} 토큰")
print(f"요약 후 최대 답변 길이: {df['output_length'].max()} 토큰")


512 토큰 초과 질문 개수: 0개
512 토큰 초과 답변 개수: 0개
요약 후 512 토큰 이하 질문 개수: 13204개
요약 후 512 토큰 이하 답변 개수: 13204개
요약 후 512 토큰 초과 질문 개수: 0개
요약 후 512 토큰 초과 답변 개수: 0개
요약 후 최대 질문 길이: 459 토큰
요약 후 최대 답변 길이: 504 토큰


# 요약 전후 비교

In [38]:
import random

# 512 토큰 초과 데이터 존재 여부 확인
if df[df["output_length"] > 512].empty:
    print("⚠ 512 토큰 초과 데이터가 없습니다. 랜덤 샘플을 선택합니다.")
    long_output_sample = df.sample(1)  # 임의의 데이터 샘플 선택
else:
    long_output_sample = df[df["output_length"] > 512].sample(1)  # 512 초과 데이터 샘플 선택

# 해당 문장의 원본과 요약 후 데이터를 가져옴
original_text = long_output_sample["output"].values[0]
original_length = long_output_sample["output_length"].values[0]

# 요약 적용 후 해당 데이터
summarized_text = summarize_text(original_text)
summarized_length = get_token_length(summarized_text)

# 결과 출력
print(f"**요약 전 텍스트:**\n{original_text}\n")
print(f"**요약 전 길이:** {original_length} 토큰\n")
print("=" * 80)
print(f"**요약 후 텍스트:**\n{summarized_text}\n")
print(f"**요약 후 길이:** {summarized_length} 토큰\n")


⚠ 512 토큰 초과 데이터가 없습니다. 랜덤 샘플을 선택합니다.
**요약 전 텍스트:**
사우님은 현재 회사에서 일하는 것에 대해 불만이 많으며, 회사의 경영진 변경으로 업무분장이 바뀌어 업무가 꼬여 업무량이 증가하였으며, 규정의 변화로 불안감과 스트레스가 많이 발생하고 있습니다. 또한 팀장님의 행동으로 팀원들이 신뢰감을 느끼지 못하고 일이 잘 안되고, 이로 인해 팀원들의 스트레스와 지장이 생기고 있습니다. 사우님은 일하는 것에 대한 불만이 많고, 이에 대한 스트레스를 느끼고 있습니다. 불안감과 스트레스로 인해 집중력이 떨어지며, 업무가 잘 안되고 있는 것 같습니다. 팀장님의 행동으로 인해 팀원들의 신뢰감을 잃게 되어, 팀원들이 스트레스를 느끼고 있습니다. 사우님은 기본적으로 일하는 것에 대한 불만이 많은 상태입니다. 현재의 직장이 맞는 직장인지, 이 직장에서 계속 일하고 싶은지, 다른 일을 해보는 것이 어떨지 등 고민을 해보는 것도 좋은 방법입니다. 만약 이 직장에서 일을 계속 하시려면, 각자 맡은 업무에 대해 정확하게 파악하고, 팀원들과의 소통을 통해 업무를 협력적으로 진행하는 것이 중요합니다. 또한 팀장님과 소통을 통해 업무분장을 다시 검토하는 것도 좋을 것 같습니다. 불안감과 스트레스로 인해 집중력이 떨어질 때는 자신의 기분을 관찰하고 적절한 타이밍에 적당한 쉬는 시간을 가져보시는 것도 좋습니다. 추가적으로 업무에 대한 스트레스를 줄이기 위해 스트레스를 해소할 수 있는 활동을 찾아보시고, 스트레스 관리를 위한 충분한 수면과 운동, 건강한 식습관 등을 유지하는 것이 중요합니다. 사우님의 상황에서는 전문가의 도움을 받아 대인관계, 스트레스, 갈등 해소 등 다양한 문제를 함께 해결하는 것이 필요합니다. 위의 답변이 사우님께 도움이 되었기를 바랍니다. 이후에도 힘든 상황이 계속되시면 언제든지 저를 찾아주세요. 좋은 하루 보내세요 :)

**요약 전 길이:** 318 토큰

**요약 후 텍스트:**
사우님은 현재 회사에서 일하는 것에 대해 불만이 많고, 업무분

# 요약 후에도 512 토큰을 초과하는 문장 자르기 (최종 정리)

In [ ]:
# # 512 토큰 초과하는 문장 잘라서 저장
# df["input"] = df["input"].apply(lambda x: tokenizer_summarization.decode(tokenizer_summarization.encode(x, add_special_tokens=True)[:512]))
# df["output"] = df["output"].apply(lambda x: tokenizer_summarization.decode(tokenizer_summarization.encode(x, add_special_tokens=True)[:512]))

# # 최종 토큰 길이 확인
# df["input_length"] = df["input"].apply(get_token_length)
# df["output_length"] = df["output"].apply(get_token_length)

# print(f"✅ 최종적으로 512 토큰 이하 질문 개수: {len(df[df['input_length'] <= 512])}개")
# print(f"✅ 최종적으로 512 토큰 이하 답변 개수: {len(df[df['output_length'] <= 512])}개")


# 최종 데이터 저장

In [ ]:
clean_summarized_csv_path = "/content/drive/MyDrive/Project/project-AI-Pet-Robot/total_kor_counsel_bot_clean_summarized.csv"
df.to_csv(clean_summarized_csv_path, index=False, encoding="utf-8")

print(f"✅최종 정제된 데이터 저장 완료 (경로: {clean_summarized_csv_path})")